In [1]:
####
# Importing Libarires


# Some links for documentation on packages
# https://python-docx.readthedocs.io/en/latest/user/documents.html
# https://docxtpl.readthedocs.io/en/latest/
# https://www.rikvoorhaar.com/python-docx/

# Installing docx in conda
# conda install docxtpl --channel conda-forge
from docx import Document, text
from docx.shared import Inches

import datetime

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import seaborn as sns
from dateutil.relativedelta import relativedelta

# Set Seaborn syle and colour pallete (to match AB gov standards)
sns.set(font="Arial")
sns.set_theme(style="white",
             palette = sns.color_palette([(0,170/255,210/255)],1),
             color_codes = True
             )
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 2
plt.rcParams['xtick.major.width'] = 0.75


# In[2]:


# List of pargrapghs with errors
para_errors = []


# In[3]:


####
# Function Definitons

# remove_front_space removes any blank spaces in front of a string, up to 20
def remove_front_space(x):
    for i in range(1,20):
        if x[0] == ' ':
            x = x[1:]
    return x

# reduce length reduces the lenght of a string to 30 characters if the string is over 38 characters
# this was done to align with prior charts
def reduce_length(x):
    if len(x)>38:
        x = x[:30] + "..."
        #x = x[:35]+"..."
    return x

# num_to_to_string takes a float or int, rounds it, multiplies by 1000, and then adds "," every 3 digits
def num_to_string(x):
    if x<0:
        x = str(int(round(x,1)*1000))[1:]
        if len(x) > 3:
            x = x[0:(len(x)-3)]+','+x[-3:]
        if len(x) > 7:
            x = x[0:(len(x)-7)]+','+x[-7:]
        x = '-'+x
        return x
    else:
        x = str(int(round(x,1)*1000))
        if len(x) > 3:
            x = x[0:(len(x)-3)]+','+x[-3:]
        if len(x) > 7:
            x = x[0:(len(x)-7)]+','+x[-7:]
        return x

# Returns different strings depending on the signage of the change
def pos_neg_zero_dem_change(number):
    if number[0] == "-":
        return " ("+number+")"
    elif number[0] == "0":
        return ""
    else:
        return " (+"+number+")"    

# Demograph_change calculates the changes for different demographic groups
def demograph_change(dem_group_list, list_type):
    if list_type == "inc":
        word_change = "increased"
    elif list_type == "dec":
        word_change = "shrunk"
    else:
        word_change = "was unchanged"
        

    # Find number of change
    if len(dem_group_list) > 0:
        # If only one change
        if len(dem_group_list) == 1:
            sentence = "Employment "+ word_change +" for "+ dem_group_list[0] + pos_neg_zero_dem_change(dems_dict[dem_group_list[0]]) + ". "
        # If multiple changes
        else:
            sentence = "Employment "+ word_change +" for "
            dem_count = 1
            for dem_group in dem_group_list:
                # End one
                if dem_count == len(dem_group_list):
                    sentence = sentence + "and "+ dem_group + pos_neg_zero_dem_change(dems_dict[dem_group])+ ". "
                # All others
                else:
                    sentence = sentence + dem_group + pos_neg_zero_dem_change(dems_dict[dem_group])+ ", "
                dem_count = dem_count + 1
    # If no changes
    else:
        sentence = ""
    
    return sentence

# Used for caclulating window sized based on width of text
def width_calculator(min_max):
    if min_max > 1000:
        min_max_lim = min_max
    elif 800< x_max < 1000:
        min_max_lim = min_max *1.5
    elif 400< x_max < 8000:
        min_max_lim = min_max *1.7
    else:
        min_max_lim = min_max *2
    return min_max_lim


def chart_text(x):
    x = num_to_string(x)
    if x[0] == "-":
        x = x[1:]
    if len(x) == 1:
        return 0.25    
    if len(x) == 2:
        return 0.375
    if len(x) == 3:
        return 0.5
    if len(x) == 5:
        return 0.625
    if len(x) == 6:
        return 0.75
    if len(x) == 7:
        return 0.825
    if len(x) == 9:
        return 1
    
def file_date_text(x):
    if len(str(x.month)) == 1:
        month = "0"+str(x.month)
    else:
        month = str(x.month)
    return str(x.year)+"-"+month



# In[4]:


# Load in the template
document = Document('Indigenous Highlights - Template.docx')


# In[5]:


####
# Dates
# Calculate text for last month and the month before

# Date time object of last month
m1 = datetime.date.today()
m1 = m1.replace(
                year=m1.year if m1.month > 1 else m1.year - 1,
                month=m1.month - 1 if m1.month > 1 else 12,
                day=1
)

# Date time object of month before
m2 = datetime.date.today()- relativedelta(months=2)

# Convert date time objects into a string for text
cur_m_text = m1.strftime('%B %Y')
pre_m_text = m2.strftime('%B %Y')


# In[6]:


# Change Metadata of template

footer_date = "©"+ str(datetime.date.today().year)+" Government of Alberta  |  "+datetime.date.today().strftime("%B %#d, %Y")+"  |  Jobs, Economy and Trade"

document.sections[0].footer.paragraphs[1].text = footer_date
document.core_properties.title = "Labour Force Statistics - Highlights: Indigenous Peoples living off reserve "+ cur_m_text
document.core_properties.author = "Government of Alberta - Jobs, Economy and Trade"
document.core_properties.subject = "Alberta Indigneous Labour Force Statistics, Employment, Unemployment, Unemployment Rate, First Nations, Métis, Metis, Indigenous People Living Off Reserve, Occupation, Industry, " + cur_m_text

# In[7]:


#### Dataframes

# Load Alberta Main excel sheet
df_main_Alberta = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'Alberta Main', header = 1)
df_main_Alberta["Date"] = df_main_Alberta['Unnamed: 1']
df_main_Alberta  = df_main_Alberta[list(df_main_Alberta.columns)[2:]]

# Find different months of data
this_month_data = df_main_Alberta.tail(1)
last_month_data = df_main_Alberta.tail(2).head(1)
last_year_data = df_main_Alberta.tail(13).head(1)


# In[8]:


# Load Canada Main excel sheet
df_main_Can = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'Canada Main', header = 1)
df_main_Can["Date"] = df_main_Can['Unnamed: 1']
df_main_Can  = df_main_Can[list(df_main_Can.columns)[2:]]

# Find different months of data
this_month_Can_data = df_main_Can.tail(1)
last_month_Can_data = df_main_Can.head(1)


# In[9]:


# Load Alberta Main - nonIndig excel sheet
df_Alberta_non_indig = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'Alberta Main Total', header = 1) 


# In[10]:


# Load ER excel sheet
df_er = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'ER', header = 1) 


# In[11]:


# Load OCC excel sheet
df_occ = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'OCC', header = 1)


# In[12]:


# Load IND excel sheet
df_ind = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'IND', header = 1)


# In[13]:


# Load Different Indigneous Main exceel sheet
df_diff_indig = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'Diff Indig Main', header = 1)


# In[14]:


# Load Age and Gender Main exceel sheet
df_age_gender = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'Age and Gender Main', header = 1)


# In[15]:


# Load COW excel sheet
df_cow = pd.read_excel("LFS Indigenous Data.xlsx", sheet_name= 'COW', header = 1)


# In[16]:


# Change Title
document.tables[0].cell(1,1).paragraphs[1].text = cur_m_text


# In[17]:


# First paragraph
try:
    # Calculate month to month change
    m_m_change = round(this_month_data["Unemployment rate"].iloc[0] - last_month_data["Unemployment rate"].iloc[0],1)

    # Change text depending on if the unemployment rate increased/decreased
    if m_m_change < 0:
        m_m_change_text = 'down'
    elif m_m_change > 0:
        m_m_change_text = 'up'
    else:
        m_m_change_text = 'the same unemployment rate as'

    # Calculate year to year change
    y_y_change = round(this_month_data["Unemployment rate"].iloc[0] - last_year_data["Unemployment rate"].iloc[0],1)

    # Change text depending on if the unemployment rate increased/decreased
    if y_y_change < 0:
        y_y_change_text = 'down'
    elif y_y_change > 0:
        y_y_change_text = 'up'
    else:
        y_y_change_text = 'the same unemployment rate as'    


    # First sentence
    first_para_first_sent = 'The unadjusted three-month moving average unemployment rate for Indigenous Peoples living off reserve in Alberta was '+str(this_month_data['Unemployment rate'].iloc[0])+ ' per cent in '+cur_m_text+'. '

    # Change text depending on if the m/m unemployment rate increased/decreased
    if float(m_m_change) == 0.0:
        first_para_secon_sent_pt1 = 'This is ' + m_m_change_text +' '+pre_m_text +' and '
    else:
        if abs(m_m_change) == 1.0:
            if m_m_change < 0:
                first_para_secon_sent_pt1 = 'This is '+ m_m_change_text+ ' ' + str(m_m_change)[1:] + ' percentage point from ' +pre_m_text + ' and ' 
            else:
                first_para_secon_sent_pt1 = 'This is '+ m_m_change_text+ ' ' + str(m_m_change) + ' percentage point from ' +pre_m_text + ' and ' 
        else:
            if m_m_change < 0:
                first_para_secon_sent_pt1 = 'This is '+ m_m_change_text+ ' ' + str(m_m_change)[1:] + ' percentage points from ' +pre_m_text + ' and ' 
            else:
                first_para_secon_sent_pt1 = 'This is '+ m_m_change_text+ ' ' + str(m_m_change) + ' percentage points from ' +pre_m_text + ' and ' 

    # Change text depending on if the y/y unemployment rate increased/decreased
    if float(y_y_change) == 0.0:    
        first_para_secon_sent_pt2 = y_y_change_text + ' the same month last year. '
    else:
        if abs(y_y_change) == 1.0:
            if y_y_change < 0:
                first_para_secon_sent_pt2 =  y_y_change_text+ ' ' + str(y_y_change)[1:] +' percentage point from the same month last year. '
            else:
                first_para_secon_sent_pt2 =  y_y_change_text+ ' ' + str(y_y_change) +' percentage point from the same month last year. '
        else:
            if y_y_change < 0:
                first_para_secon_sent_pt2 =  y_y_change_text+ ' ' + str(y_y_change)[1:] +' percentage points from the same month last year. '
            else:
                 first_para_secon_sent_pt2 =  y_y_change_text+ ' ' + str(y_y_change)+' percentage points from the same month last year. '


    # Form sentence
    first_para_secon_sent = first_para_secon_sent_pt1 + first_para_secon_sent_pt2    

    # Canadian unemployment rate formating and change
    can_rate = round(this_month_Can_data["Unemployment rate"].iloc[0],1)
    can_change = round(this_month_Can_data["Unemployment rate"].iloc[0] - last_month_Can_data["Unemployment rate"].iloc[0],1)

    # Change text depending on if the unemployment rate increased/decreased
    if abs(can_change) == 1.0:
        if can_change < 0:
            first_para_third_sent = 'The national rate was '+str(can_rate)+' per cent, a decrease of '+str(can_change)[1:]+' percentage point from the month before. '
        elif can_change >0:
            first_para_third_sent = 'The national rate was '+str(can_rate)+' per cent, an increase of '+str(can_change)+' percentage point from the month before. '
        else:
            first_para_third_sent = 'The national rate was '+str(can_rate)+' per cent, unchanged from the month before. '
    else:
        if can_change < 0:
            first_para_third_sent = 'The national rate was '+str(can_rate)+' per cent, a decrease of '+str(can_change)[1:]+' percentage points from the month before. '
        elif can_change >0:
            first_para_third_sent = 'The national rate was '+str(can_rate)+' per cent, an increase of '+str(can_change)+' percentage points from the month before. '
        else:
            first_para_third_sent = 'The national rate was '+str(can_rate)+' per cent, unchanged from the month before. '

    # Form paragraph
    document.paragraphs[2].text = first_para_first_sent + first_para_secon_sent + first_para_third_sent
except:
    para_errors.append("paragraph 1")


# In[18]:


# Chart 1

try:
    # Initialize the matplotlib figure
    # Size of plot
    f, ax = plt.subplots(figsize=(3.28, 2.22))
    # Chart title
    ax.set_title('Unemployment Rate\nUnadjusted, Three-Month Moving Average', size = 8, color = 'black')
    # Set size of y and x axis labels
    ax.tick_params(axis='y', which='major', labelsize=7)
    ax.tick_params(axis='x', which='major', labelsize=5)
    # Create chart
    chart = sns.lineplot(x = "Date", y = "Unemployment rate", data=df_main_Alberta)
    # Axis titles
    ax.set( ylabel="",
           xlabel="")
    # Format y axis with %
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    # Interval betweeen months
    locator = mdates.MonthLocator(interval=6)
    chart.xaxis.set_major_locator(locator)
    # Remove graph borders
    sns.despine(left=True)
    # Save plot with high resolution
    plt.savefig("Indig line.png",bbox_inches='tight', dpi = 1200)
    plt.close()



    # Add saved plot to document
    p = document.paragraphs[3]
    r = p.add_run()
    r.add_picture('Indig line.png')

except:
    para_errors.append("chart 1")


# In[19]:


# Second paragraph
try:
    # Part conditional on unemployment rate
    if m_m_change_text == 'down':
        second_para_inc_dec_1 = 'decreased'
    elif m_m_change_text == 'up':
        second_para_inc_dec_1 = 'increased'
    else:
        second_para_inc_dec_1 = 'remained the same'

    # Calculate m/m change in labour force
    labf_m_m_change = round(this_month_data['  Labour force'].iloc[0] - last_month_data['  Labour force'].iloc[0],1)

    # Part conditional on labour force, make change to string
    if labf_m_m_change < 0:
        second_para_inc_dec_2 = 'decreased'
        labf_m_m_change = num_to_string(labf_m_m_change)[1:]
    elif labf_m_m_change > 0:
        second_para_inc_dec_2 = 'increased'
        labf_m_m_change = num_to_string(labf_m_m_change)

    else:
        second_para_inc_dec_2 = 'remained the same'   

    # Form sentence    
    if second_para_inc_dec_2 == 'remained the same':
        second_para_pt1 = 'The unemployment rate '+second_para_inc_dec_1+' as the labour force '+ second_para_inc_dec_2 +' and ' 
    else:
        second_para_pt1 = 'The unemployment rate '+second_para_inc_dec_1+' as the labour force '+ second_para_inc_dec_2 +' by '+ labf_m_m_change+' and ' 

    # Calculate m/m change in employment    
    empl_m_m_change = round(this_month_data['    Employment'].iloc[0] - last_month_data['    Employment'].iloc[0],1)
    # Part conditional on employment, make change to string
    if empl_m_m_change < 0:
        second_para_inc_dec_3 = 'decreased'
        empl_m_m_change = num_to_string(empl_m_m_change)[1:]
    elif empl_m_m_change > 0:
        second_para_inc_dec_3 = 'increased'
        empl_m_m_change = num_to_string(empl_m_m_change)
    else:
        second_para_inc_dec_3 = 'remained the same'

    # Form sentence  
    if second_para_inc_dec_3 == 'remained the same':
        second_para_pt2 = 'employment '+ second_para_inc_dec_3 +' from '+pre_m_text+' to ' + cur_m_text +'.'
    else:
        second_para_pt2 = 'employment '+ second_para_inc_dec_3 +' by '+empl_m_m_change+' from '+pre_m_text+' to ' + cur_m_text +'.'

    # Form paragraph    
    document.paragraphs[6].text = second_para_pt1 + second_para_pt2
except:
    para_errors.append("paragraph 2")  


# In[20]:


# Third Paragraph
try:
    third_para_sent1_part1 = "In " + cur_m_text + ", the unemployment rates for First Nations and Métis people were "
    third_para_sent1_part2 = str(df_diff_indig["      First Nations"].iloc[0]) + " per cent and "+str(df_diff_indig["      Métis"].iloc[0]) + " per cent respectively."
    document.paragraphs[8].text = third_para_sent1_part1 + third_para_sent1_part2

except:
    para_errors.append("paragraph 3")  


# In[21]:


# Fourth paragraph

try:
    # Calculate percentage of total employment
    pct = str(round((this_month_data['    Employment'].iloc[0]/df_Alberta_non_indig['    Employment'].iloc[0])*100,1))

    # Convert employment, part-time employment, and full-time employment into strings
    ind_employment = num_to_string(this_month_data['    Employment'].iloc[0])        
    ind_pt_employment = num_to_string(this_month_data['       Part-time employment'].iloc[0])
    ind_ft_employment = num_to_string(this_month_data['       Full-time employment'].iloc[0])    

    # Form sentences
    fourth_para_first_sent = 'In '+cur_m_text +', there were '+ind_employment+' employed Indigenous Peoples living off reserve. '
    fourth_para_secon_sent = 'This represents '+ pct+' per cent of Alberta’s overall employment. '
    fourth_para_third_sent = 'Of those employed, '+ind_ft_employment+' were employed full-time and '+ind_pt_employment+' were employed part-time.'

    # Form paragraph
    document.paragraphs[10].text = fourth_para_first_sent+fourth_para_secon_sent+fourth_para_third_sent

except:
    para_errors.append("paragraph 4")


# In[22]:


# Fifth paragraph

try:
    priv_employment = num_to_string(df_cow["    Private sector employees"].iloc[0])
    pub_employment = num_to_string(df_cow["    Public sector employees"].iloc[0])
    self_employment = num_to_string(df_cow[" Self-employed (including unpaid family worker)"].iloc[0])
    
    fifth_para_part_one = "Of the "+ ind_employment + " employed Indigenous Peoples living off reserve, " +priv_employment
    fifth_para_part_two = " worked in the private sector, " + pub_employment + " in the public sector, and " + self_employment + " were self-employed."

    document.paragraphs[12].text = fifth_para_part_one + fifth_para_part_two
    
except:
    para_errors.append("paragraph 5")


# In[23]:


# Sixth paragraph

try:
    # Calculate employment by economic region. Convert format to string
    er_Alberta = num_to_string(df_er[ 'Alberta'].iloc[0] - df_er['4860 - Edmonton'].iloc[0] - df_er['4830 - Calgary'].iloc[0])      
    er_Edmonton = num_to_string(df_er['4860 - Edmonton'].iloc[0])
    er_Calgary = num_to_string(df_er['4830 - Calgary'].iloc[0])
    frac_in_ec = str(round(100*(df_er['4860 - Edmonton'].iloc[0] + df_er['4830 - Calgary'].iloc[0])/df_er[ 'Alberta'].iloc[0],1))
    
    # Form sentences
    sixth_para_first_sent = "In " + cur_m_text + ", " + frac_in_ec+" per cent of employment for Indigenous Peoples in the province was in the Edmonton (" +er_Edmonton+") and Calgary ("+er_Calgary+") economic regions. "
    sixth_para_secon_sent = "There were "+er_Alberta+ " employed Indigenous Peoples in the rest of Alberta."
    # Form paragraph
    document.paragraphs[14].text = sixth_para_first_sent + sixth_para_secon_sent

except:
    para_errors.append("paragraph 6")


# In[24]:


# seventh paragraph
try:
    # Calculate changes and convert to strings
    fem_15_24 = num_to_string(df_age_gender["15-24 years"].iloc[5] - df_age_gender["15-24 years"].iloc[2])
    fem_25_54 = num_to_string(df_age_gender["25-54 years"].iloc[5] - df_age_gender["25-54 years"].iloc[2])
    fem_55_up = num_to_string(df_age_gender["55 years and over"].iloc[5] - df_age_gender["55 years and over"].iloc[2])

    mal_15_24 = num_to_string(df_age_gender["15-24 years"].iloc[4] - df_age_gender["15-24 years"].iloc[1])
    mal_25_54 = num_to_string(df_age_gender["25-54 years"].iloc[4] - df_age_gender["25-54 years"].iloc[1])
    mal_55_up = num_to_string(df_age_gender["55 years and over"].iloc[4] - df_age_gender["55 years and over"].iloc[1])

    # Make text labels
    fem_15_24_text = "females aged 15 to 24 years"
    fem_25_54_text = "females aged 25 to 54 years"
    fem_55_up_text = "females aged 55+ years"

    mal_15_24_text = "males aged 15 to 24 years"
    mal_25_54_text = "males aged 25 to 54 years"
    mal_55_up_text = "males aged 55+ years"


    # List of the labels
    dems_list = [fem_15_24_text,
                 fem_25_54_text,
                 fem_55_up_text,
                 mal_15_24_text,
                 mal_25_54_text,
                 mal_55_up_text] 

    # Dictionary tying label to change
    dems_dict = {fem_15_24_text : fem_15_24,
                 fem_25_54_text : fem_25_54,
                 fem_55_up_text : fem_55_up,
                 mal_15_24_text : mal_15_24,
                 mal_25_54_text : mal_25_54,
                 mal_55_up_text : mal_55_up}

    # make lists for the types of changes
    dem_increases = []
    dem_decreases = []
    dem_no_change = []

    # Sort lables
    for dem in dems_list:
        if dems_dict[dem][0] == '-':
            dem_decreases.append(dem)
        elif dems_dict[dem][0] == '0':
            dem_no_change.append(dem)
        else:
            dem_increases.append(dem)

    seventh_para_sent_one = demograph_change(dem_increases, "inc")
    seventh_para_sent_two =demograph_change(dem_decreases, "dec")
    seventh_para_sent_three =demograph_change(dem_no_change, "no")


    document.paragraphs[16].text = seventh_para_sent_one +seventh_para_sent_two + seventh_para_sent_three

except:
    para_errors.append("paragraph 7")


# In[25]:


# eigth paragraph

try:
    # Paragraph 8 pre-processing
    # Occupation column names
    occ_col_1 = list(df_occ.columns)[1]
    occ_col_2 = list(df_occ.columns)[2]

    # Grab top 3 occupations
    # Note that total is in table so have to grab head of tail
    occ_sorted = df_occ.sort_values(occ_col_2).tail(4).head(3)

    # Grab names of top three occupations, make lower case
    occ_first = occ_sorted[occ_col_1].iloc[2][4:].lower()
    occ_second = occ_sorted[occ_col_1].iloc[1][4:].lower()
    occ_third = occ_sorted[occ_col_1].iloc[0][4:].lower()

    # Grab values of the top three occupations, converting to string
    occ_first_val = "("+num_to_string(occ_sorted[occ_col_2].iloc[2])+")"
    occ_second_val = "("+num_to_string(occ_sorted[occ_col_2].iloc[1])+")"
    occ_third_val = "("+num_to_string(occ_sorted[occ_col_2].iloc[0])+")"

    # Write sententences of paragraph 7
    eigth_para_first_sect = 'The three occupations that employed the greatest number of Indigenous Peoples in '+cur_m_text+' were: '
    eigth_para_secon_sect = occ_first+' '+occ_first_val+'; '+occ_second+' '+occ_second_val+'; and '+occ_third+' '+occ_third_val+'.'

    # Write paragraph
    document.paragraphs[18].text = eigth_para_first_sect + eigth_para_secon_sect

except:
    para_errors.append("paragraph 8")


# In[26]:


# Ninth paragraph

try:
    # Do Industry Calculations - Paragraph 6 pre-processsing
    # Grab column names
    ind_col_1 = list(df_ind.columns)[1]
    pre_month = list(df_ind.columns)[2]
    last_month = list(df_ind.columns)[3]

    # List of industry with how it appears in Beyond 20/20 table
    # List also filters out other industries
    ind=['      Agriculture',
    '  Public administration',
    '    Educational services',
    '    Accommodation and food services',
    '      Manufacturing',
    '    Management of companies and administrative and other support services',
    '    Other services',
    '    Information, culture and recreation',
    '    Utilities',
    '    Professional, scientific and technical services',
    '      Construction',
    '    Finance, insurance, real estate and leasing',
    '    Transportation and warehousing',
    '      Forestry, fishing, mining, oil and gas',
    '    Health care and social assistance',
    '    Trade']

    # Filter out industries
    df_ind = df_ind[df_ind[ind_col_1].isin(ind)]
    # Calculate monthly change
    df_ind["Change"] = df_ind[last_month] - df_ind[pre_month]
    # Sort by month to month change
    df_ind_sorted = df_ind.sort_values("Change", ascending = False)
    # Remove excess space in industry strings
    df_ind_sorted[ind_col_1] = df_ind_sorted[ind_col_1].apply(remove_front_space)

    # Paragraph 6
    # Grab names of top three industries, make lower case
    ind_first = df_ind_sorted.head(3)[ind_col_1].iloc[0].lower()
    ind_second = df_ind_sorted.head(3)[ind_col_1].iloc[1].lower()
    ind_third = df_ind_sorted.head(3)[ind_col_1].iloc[2].lower()
    # Grab values of the top three industries, converting to string
    ind_first_val = num_to_string(df_ind_sorted.head(3)["Change"].iloc[0])
    ind_second_val = num_to_string(df_ind_sorted.head(3)["Change"].iloc[1])
    ind_third_val = num_to_string(df_ind_sorted.head(3)["Change"].iloc[2])

    ind_counter = 0
    if ind_first_val[0] != "-" and ind_first_val[0] != "0":
        ind_counter = ind_counter + 1
    if ind_second_val[0] != "-" and ind_first_val[0] != "0":
        ind_counter = ind_counter + 1
    if ind_third_val[0] != "-" and ind_first_val[0] != "0":
        ind_counter = ind_counter + 1
    
    # Form sentences
    if ind_counter == 0:
        ninth_para_first_sect = 'Between '+ pre_m_text +' and '+cur_m_text+', there were no industries with employment increases.'
        ninth_para_secon_sect = ""
    elif ind_counter == 1:
        ninth_para_first_sect = 'Between '+ pre_m_text +' and '+cur_m_text+', the only industry where employment increased was the '+ ind_first+' industry.'
        ninth_para_secon_sect = ""
    elif ind_counter == 2:   
        ninth_para_first_sect = 'Between '+ pre_m_text +' and '+cur_m_text+', the industries with the most employment increases were: '
        ninth_para_secon_sect = ind_first+'; and '+ind_second+'. '
    else:
        ninth_para_first_sect = 'Between '+ pre_m_text +' and '+cur_m_text+', the industries with the most employment increases were: '
        ninth_para_secon_sect = ind_first+'; '+ind_second+'; and '+ind_third+'. '

    # Form paragraph
    document.paragraphs[20].text = ninth_para_first_sect + ninth_para_secon_sect

except:
    para_errors.append("paragraph 9")


# In[27]:


# Chart 2

try:
    chart2_pos = 0.2
    # Chart 2 pre-processing
    df_ind_sorted["Change"] = df_ind_sorted["Change"]*1000
    df_ind_sorted["Display"] = df_ind_sorted[ind_col_1].apply(reduce_length)

    # Initialize the matplotlib figure
    # Plot size
    f, ax = plt.subplots(figsize=(2.2, 3.2))
    # Plot title
    ax.set_title('Employment Growth\nBy Industry\nUnadjusted, Three-Month Moving Average', size = 7, x = 0.2, color = 'black')
    # Make chart
    sns.barplot(x="Change", y="Display", data=df_ind_sorted,palette = sns.color_palette([(0,170/255,210/255)],1),
               edgecolor = (0,0,0))
    # Axis title
    ax.set( ylabel="",
           xlabel="")
    # Label formatting
    ax.tick_params(axis='both', which='major', labelsize=6)
    # Remove chart borders
    sns.despine(left=True, bottom=True)
    # Save figure in high resolution

    #Set x width
    x_max = df_ind_sorted['Change'].max()
    x_min = df_ind_sorted['Change'].min()

    ####

    if df_ind_sorted['Change'].max() >0:
        x_max = df_ind_sorted['Change'].max()+chart_text(df_ind_sorted['Change'].max()/1000)/chart2_pos+ 0.125/chart2_pos
    else:
        x_max = 0
    if df_ind_sorted['Change'].min() <0:
        x_min = df_ind_sorted['Change'].min()-chart_text(df_ind_sorted['Change'].min()/1000)/chart2_pos-0.25/chart2_pos
    else:
        x_min = 0
    ax.set_xlim(left = -abs(x_min)**1.06, right = x_max**1.06)

    chart2_pos2 = 8/(x_max-x_min)

    # Add labels
    for p in ax.patches:
        if p.get_width()<0:
            ax.text(p.get_width()-(chart_text(p.get_width()/1000)/chart2_pos2)- 0.25/chart2_pos2,
                    p.get_y()+ p.get_height()*0.75, num_to_string(int(p.get_width())/1000), 
                fontsize=5, ha='center', va='bottom')
        else:
            ax.text(p.get_width()+(chart_text(p.get_width()/1000)/chart2_pos2)+ 0.125/chart2_pos2,
                    p.get_y()+ p.get_height()*0.75, num_to_string(int(p.get_width())/1000), 
                fontsize=5, ha='center', va='bottom')

    plt.setp(ax.patches, linewidth=0.5)        
    ax.axvline(0, color = (209/255, 212/255,211/255), linewidth=1)   



    #####


    ax.set_xlim([width_calculator(x_min), width_calculator(x_max)])

    # Remove ticks and labels
    ax.set(xticklabels=[])
    ax.tick_params(bottom=False)





    plt.savefig("Indig bar.png",bbox_inches='tight', dpi = 1200) 
    plt.close()

    # Add saved plot to document
    q = document.paragraphs[21]
    s = q.add_run()
    s.add_picture('Indig bar.png')



except:
    para_errors.append("chart 2")


# In[28]:


# Save document with month
name = 'jet-lfs-indigenous-highlights-'+file_date_text(m1)+'.docx'
document.save(name)
print("Program has been run. " +name+" is now available.")


# In[29]:


number_of_errors = len(para_errors)
if number_of_errors > 0:
    print_errors = 'The following paragraphs/charts have errors:'
    if number_of_errors != 1:
        error_count = 0
        for para in para_errors:
            error_count = error_count + 1
            if error_count == number_of_errors:
                print_errors = print_errors +" and "+ para +"."
            else:
                print_errors = print_errors +" " +para +","
    else:
        print_errors = '\nThe following paragraph/chart has errors: ' + para_errors[0]+"."
    print_errors = print_errors + "\nPlease do the listed paragraphs/charts manually before sending to comms.\n\nAddionally, please double check the numbers and language in the other paragraphs."
else:
    print_errors = "\nThere were no errors in the paragraphs. Please double check the numbers, the language, and send the document to comms."

print(print_errors)



Program has been run. jet-lfs-indigenous-highlights-2024-07.docx is now available.

There were no errors in the paragraphs. Please double check the numbers, the language, and send the document to comms.
